# NLP Code Along

For this code along we will build a spam filter! We'll use the various NLP tools we learned about as well as a new classifier, Naive Bayes.

We'll use a classic dataset for this - UCI Repository SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [3]:
data = spark.read.csv("data/smsspamcollection/SMSSpamCollection",inferSchema=True,sep='\t')

In [4]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [7]:
data.count()

5574

## Clean and Prepare the Data

** Create a new length feature: **

In [8]:
from pyspark.sql.functions import length

In [9]:
data = data.withColumn('length',length(data['text']))

In [10]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [12]:
# Pretty Clear Difference
data.groupby('class').mean('length').show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

In [13]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer

tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [14]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [15]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

### The Model

We'll use Naive Bayes, but feel free to play around with this choice!

In [16]:
from pyspark.ml.classification import NaiveBayes

In [17]:
# Use defaults
nb = NaiveBayes()

### Pipeline

In [18]:
from pyspark.ml import Pipeline

In [19]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

cleaner = data_prep_pipe.fit(data)

clean_data = cleaner.transform(data)

### Training and Evaluation!

In [22]:
clean_data = clean_data.select(['label','features'])

In [25]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [26]:
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [27]:
spam_predictor = nb.fit(training)

In [28]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [29]:
test_results = spam_predictor.transform(testing)

In [30]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,5,20,...|[-815.16152870799...|[1.0,3.8155166809...|       0.0|
|  0.0|(13424,[0,1,11,32...|[-887.22051798021...|[1.0,1.1740794818...|       0.0|
|  0.0|(13424,[0,1,14,31...|[-216.86469213215...|[1.0,1.8043035131...|       0.0|
|  0.0|(13424,[0,1,18,20...|[-835.50521877327...|[1.0,6.7668440205...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-969.59733322870...|[0.99999999999999...|       0.0|
|  0.0|(13424,[0,1,23,63...|[-1310.2038199509...|[1.0,1.4445273727...|       0.0|
|  0.0|(13424,[0,1,27,88...|[-1528.3283033350...|[2.81586176771694...|       1.0|
|  0.0|(13424,[0,1,30,12...|[-599.71537657558...|[1.0,3.3380034363...|       0.0|
|  0.0|(13424,[0,1,46,17...|[-1142.0664481657...|[8.10790325150864...|       1.0|
|  0.0|(13424,[0

In [31]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [32]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.9237889484309704
